In [6]:
from pandas import set_option as pd_set_option
pd_set_option('display.max_rows', 500)
pd_set_option('display.max_columns', 500)

from excel_app import getDataFromExcel
from pandas import concat as pd_concat
from pandas import merge as pd_merge

def get_xls_struct():
    dates_pivot = {
        "Период": "date",
        "ДоговорСсылка": "treat",
        "Филиал": "branch",
        "ОплатаЗаПериодПлан": "pay_plan",
        "ОплатаЗаПериодФакт": "pay_fact",
        "СуммаЗаПериодПлан": "revenue_plan",
        "ВыручкаЗаПериодФакт": "revenue_fact"}

    revenue = {
        "Филиал": "branch",
        "ДоговорСсылка": "treat",
        "Дох_Договор": "rev_treat",
        "Дох_Контрагент": "treat_client",
        "Дох_ИНН": "treat_inn",
        "Дох_НомерДоговора": "treat_num",
        "Дох_Дата": "treat_date",
        "Дох_ДатаОкончания": "treat_end_date",
        "Дох_ОбъектДоговора": "treat_object",
        "Дох_Тип": "treat_type",
        "СуммаДоговора": "treat_amount",
        "СуммаНачало": "amount_done_on_begin",
        "ОстатокНачало": "amount_on_begin",
        "ОплатаНаНачало": "pay_on_begin",
        "СуммаКонец": "amount_done_on_end",
        "ОстатокКонец": "amount_on_end"}

    expense = {
        "Филиал": "branch",
        "ДоговорСсылка": "treat",
        "Дох_Договор": "rev_treat",
        "Расх_Договор": "exp_treat",
        "Расх_Тип": "exp_type",
        "Расх_Контрагент": "exp_client",
        "Расх_ИНН": "exp_inn",
        "Расх_НомерДоговора": "exp_num",
        "Расх_Дата": "exp_date",
        "СуммаДоговора": "treat_amount",
        "СуммаНачало": "amount_done_on_begin",
        "ОстатокНачало": "amount_on_begin",
        "СуммаКонец": "amount_done_on_end",
        "ОстатокКонец": "amount_on_end"}

    list_rev = list(revenue.values())
    for exp_var in list(expense.values()):
        if exp_var not in list_rev:
            list_rev.append(exp_var)

    return [dates_pivot, revenue, expense, list_rev]

def get_contract_report():
    xls_struct = get_xls_struct()
    df_period = getDataFromExcel("files//periods.xlsx", list(xls_struct[0]))
    df_period = df_period.rename(columns=xls_struct[0])
    df_rev = getDataFromExcel("files//revenew.xlsx", list(xls_struct[1]))
    df_rev = df_rev.rename(columns=xls_struct[1])
    df_exp = getDataFromExcel("files//expense.xlsx", list(xls_struct[2]))
    df_exp = df_exp.rename(columns=xls_struct[2])

    result = pd_concat([df_rev, df_exp], ignore_index=True, sort=False)
    result = result.loc[result['rev_treat'] == '"Договор 00000000052 от 25.01.2012 13:03:37"']
    result = pd_merge(result, df_period, on=["treat", "branch"], how="left")
    result = result.fillna(
        {"date": "01.01.2019 0:00:00", "pay_plan": 0, "pay_fact": 0, "revenue_plan": 0, "revenue_fact": 0}).fillna(
        "null")
    result = result.pivot_table(index=xls_struct[3], columns="date",
                                values=["pay_plan", "pay_fact", "revenue_plan", "revenue_fact"], fill_value=0,
                                margins=True,
                                aggfunc=sum)
    # result = result.swaplevel(0, 1, axis=1).sort_index(axis=1)
    result = result.reset_index()
    return result

get_contract_report()

branch                                         treat  \
date                                                                        
0     Центральный офис (ЦО)  "Договор 00000000024 от 22.12.2009 12:00:01"   
1     Центральный офис (ЦО)  "Договор 00000000052 от 25.01.2012 13:03:37"   
2     Центральный офис (ЦО)  "Договор 00000000172 от 13.02.2012 10:48:20"   
3     Центральный офис (ЦО)  "Договор 00000000447 от 01.06.2007 14:31:21"   
4     Центральный офис (ЦО)  "Договор 00000000452 от 29.10.2010 14:48:47"   
5     Центральный офис (ЦО)  "Договор 00000000457 от 09.12.2010 14:58:28"   
6     Центральный офис (ЦО)  "Договор 00000000466 от 01.05.2010 15:19:47"   
7     Центральный офис (ЦО)   "Договор 00000000654 от 09.06.2004 8:49:22"   
8     Центральный офис (ЦО)   "Договор 00000000660 от 29.12.2012 9:57:11"   
9     Центральный офис (ЦО)  "Договор 00000000905 от 01.02.2010 16:06:25"   
10    Центральный офис (ЦО)  "Договор 00000000991 от 22.08.2012 13:59:39"   
11    Центральный офис (ЦО)  "Договор 00000001258 от 27.03.2013 12:00:00"   
12    Центральный офис (ЦО)   "Договор 00000001279 от 11.04.2013 0:00:00"   
13    Центральный офис (ЦО)  "Договор 00000001385 от 26.06.2013 12:00:00"   
14    Центральный офис (ЦО)  "Договор 00000001571 от 12.09.2013 12:00:00"   
15    Центральный офис (ЦО)  "Договор 00000001894 от 06.11.2013 12:00:02"   
16    Центральный офис (ЦО)  "Договор 00000003129 от 29.05.2015 12:00:00"   
17    Центральный офис (ЦО)  "Договор 00000003156 от 08.06.2015 12:00:01"   
18    Центральный офис (ЦО)  "Договор 00000003205 от 23.08.2015 12:00:01"   
19    Центральный офис (ЦО)  "Договор 00000003212 от 24.07.2015 12:00:00"   
20    Центральный офис (ЦО)  "Договор 00000003228 от 24.07.2015 12:00:04"   
21    Центральный офис (ЦО)  "Договор 00000003339 от 26.05.2008 12:00:00"   
22    Центральный офис (ЦО)  "Договор 00000003342 от 19.01.2015 12:00:00"   
23    Центральный офис (ЦО)  "Договор 00000003345 от 01.07.2015 12:00:00"   
24    Центральный офис (ЦО)   "Договор 00000003362 от 10.09.2009 0:00:00"   
25    Центральный офис (ЦО)  "Договор 00000003457 от 18.08.2015 12:00:01"   
26    Центральный офис (ЦО)  "Договор 00000003460 от 18.08.2015 12:00:02"   
27    Центральный офис (ЦО)  "Договор 00000003466 от 23.06.2015 12:00:01"   
28    Центральный офис (ЦО)  "Договор 00000003477 от 24.07.2015 12:00:13"   
29    Центральный офис (ЦО)  "Договор 00000003499 от 14.08.2015 12:00:03"   
30    Центральный офис (ЦО)  "Договор 00000003515 от 07.08.2015 12:00:08"   
31    Центральный офис (ЦО)  "Договор 00000003525 от 23.06.2015 12:00:05"   
32    Центральный офис (ЦО)  "Договор 00000003527 от 23.06.2015 12:00:06"   
33    Центральный офис (ЦО)  "Договор 00000003531 от 07.08.2015 12:00:10"   
34    Центральный офис (ЦО)  "Договор 00000003535 от 23.06.2015 12:00:08"   
35    Центральный офис (ЦО)  "Договор 00000003551 от 09.09.2015 12:00:00"   
36    Центральный офис (ЦО)  "Договор 00000003553 от 09.09.2015 12:00:01"   
37    Центральный офис (ЦО)  "Договор 00000003587 от 23.06.2015 12:00:09"   
38    Центральный офис (ЦО)  "Договор 00000003593 от 23.06.2015 12:00:12"   
39    Центральный офис (ЦО)  "Договор 00000003605 от 24.07.2015 12:00:18"   
40    Центральный офис (ЦО)  "Договор 00000003607 от 24.07.2015 12:00:19"   
41    Центральный офис (ЦО)  "Договор 00000003635 от 07.08.2015 12:00:22"   
42    Центральный офис (ЦО)  "Договор 00000003637 от 07.08.2015 12:00:23"   
43    Центральный офис (ЦО)  "Договор 00000003639 от 07.08.2015 12:00:24"   
44    Центральный офис (ЦО)  "Договор 00000003641 от 07.08.2015 12:00:25"   
45    Центральный офис (ЦО)  "Договор 00000003643 от 07.08.2015 12:00:26"   
46    Центральный офис (ЦО)  "Договор 00000003645 от 07.08.2015 12:00:27"   
47    Центральный офис (ЦО)  "Договор 00000003647 от 07.08.2015 12:00:28"   
48    Центральный офис (ЦО)  "Договор 00000003649 от 07.08.2015 12:00:29"   
49    Центральный офис (ЦО)  "Договор 00000003651 от 07.08.2015 12:00:30"   
50    Центральный